
# Implementing a Segmentation Model

## 1. Preparations
### 1.1 Install required packages

In [6]:
import json
import os
import geopandas as gpd
import sklearn
import torch
import matplotlib.pyplot as plt
import numpy as np

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
!pip install segmentation-models-pytorch

In [5]:
import segmentation_models_pytorch as smp

ModuleNotFoundError: No module named 'torch'

### 1.2 Upload data to colab

In [ ]:
# manual upload (for small files)
# from google.colab import files
# uploaded = files.upload()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Data directories on GOOGLE DRIVE  # -------------------->> ADJUSTABLE
# npy patches
patches_dir = '/content/drive/My Drive/Dokumente.GD/FS06 SS24/BACHELORARBEIT/MoSE/data/patches'
# JSON class codes
codes_dir = '/content/drive/My Drive/Dokumente.GD/FS06 SS24/BACHELORARBEIT/MoSE/data/classes'
# Shapefile class labels
labels_dir = '/content/drive/My Drive/Dokumente.GD/FS06 SS24/BACHELORARBEIT/MoSE/data/shapefiles'

In [ ]:
# Class labels
# specific Shapefile path
shp_path = os.path.join(labels_dir, "GSK_24_WGS84_adjusted.shp") # -------------------->> ADJUSTABLE

labels = gpd.read_file(shp_path) # read shapefile
labels_filtered = labels[labels["Elementtyp"].notnull()] # remove NULL
print(labels_filtered["Elementtyp"].unique()) # print all label classes

In [ ]:
# Class codes
# path
label_codes_path = os.path.join(codes_dir, "label_codes.json")   # -------------------->> ADJUSTABLE

# Open and load the JSON file
with open(label_codes_path, "r") as json_file:
    label_codes = json.load(json_file)

print(label_codes)
len(label_codes)

{'Einzelstein': 1, 'Wurzelstock': 2, 'Steinverbauung': 3, 'Totholz': 4, 'Steinriegel': 5, 'Schotterbank': 6, 'Schlamm_Sandinsel': 7, 'Sand_Schlammbank': 8, 'Schotterinsel': 9}


9

In [ ]:
# Test patch


## 2. Choose segmentation model

In [ ]:
model = smp.Unet(   # -------------------->> ADJUSTABLE
    encoder_name="resnet34",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (3 for RGB)
    classes=len(label_codes),       # model output channels (number of classes)
)

NameError: name 'smp' is not defined

## 3. Configure data preprocessing

Datenvorverarbeitungsfunktion (preprocess_input) für ein bestimmtes Modell (Encoder) konfigurieren:

`from segmentation_models_pytorch.encoders import get_preprocessing_fn`

`preprocess_input = get_preprocessing_fn('resnet18', pretrained='imagenet')`

Encoder = der Teil des neuronalen Netzes, der Features aus den Eingabedaten extrahiert (in diesem Fall das vortrainiertes Modell "resnet" auf ImageNet)
Pretraining = Encoder bereits auf einem großen Datensatz (z. B. ImageNet) trainiert > kann nützliche feature erkennen

Warum Datenvorverarbeitung nötig ist:
- Während des Trainings der vortrainierten Gewichte (z. B. auf ImageNet) wurde eine spezifische Vorverarbeitung auf die Bilder angewendet, z. B. Normalisierung, Größenanpassung oder Farbraumkonvertierung.
- Um die bestmögliche Leistung mit den vortrainierten Gewichten zu erzielen, sollten die Eingabedaten genauso vorverarbeitet werden, wie es während des Pretrainings gemacht wurde.

Was `get_preprocessing_fn` macht:
Diese Funktion liefert eine Vorverarbeitungsfunktion (preprocess_input), die genau die gleichen Transformationen durchführt, die während des Pretrainings auf den ImageNet-Datensatz angewendet wurden.





In [ ]:
from segmentation_models_pytorch.encoders import get_preprocessing_fn

preprocess_input = get_preprocessing_fn('resnet18', pretrained='imagenet') # load specific (suitable to chosen model) preprocessing function

In [ ]:
# create output directory for preprocessed patches
preprocessed_patches_dir = '/content/drive/My Drive/Dokumente.GD/FS06 SS24/BACHELORARBEIT/MoSE/data/preprocessed_patches'
if not os.path.exists(preprocessed_patches_dir):
    os.makedirs(preprocessed_patches_dir)

In [ ]:
patches_list = [f for f in os.listdir(patches_dir) if not f.endswith('_mask.npy')]  # list of all patches (not the masks)
patches_list.sort() # sort list alphabetically
patches_list

for idx, patch_name in enumerate(patches_list):
    patch_path = os.path.join(patches_dir, patch_name)  # path to specific patch
    patch = np.load(patch_path)  # load npy file (Shape: [Channels, Height, Width])

# HIEEER WEITER
    # Überprüfe das Shape und transponiere, falls nötig
    if patch.shape[0] in [1, 3]:  # Annahme: Shape ist [C, H, W] oder [H, W, C]
        patch = patch.transpose(1, 2, 0)  # [H, W, C] für preprocess_input

    # Wende die Preprocessing-Funktion an
    preprocessed_patch = preprocess_input(patch)

    # Transponiere zurück zu [C, H, W], wenn nötig
    preprocessed_patch = preprocessed_patch.transpose(2, 0, 1)

    # Speichere das vorverarbeitete Patch
    output_path = os.path.join(output_dir, file_name)
    np.save(output_path, preprocessed_patch)

    # Fortschritt ausgeben
    if idx % 100 == 0:
        print(f"Processed {idx}/{len(file_list)} patches")


## 4. Write training and  testing loop

### 4.1 Choose optimizer and loss function

In [ ]:
loss = smp.losses.DiceLoss(mode="multiclass")
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)